In [2]:
import random 
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [3]:
text_df = pd.read_csv("fake_or_real_news.csv")

In [7]:
text_df.head()

,id,title,text
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...


In [6]:
text_df.drop(columns=["label"], inplace = True)

In [8]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [9]:
partial_text = joined_text[:10000]

In [10]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [11]:
tokens

['daniel',
 'greenfield',
 'a',
 'shillman',
 'journalism',
 'fellow',
 'at',
 'the',
 'freedom',
 'center',
 'is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 'but',
 'it',
 's',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 'but',
 'that',
 's',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'director',
 'james',
 'c

In [12]:
unique_tokens = np.unique(tokens)

unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [13]:
unique_token_index

{'2016': 0,
 '2020': 1,
 '5': 2,
 '60': 3,
 'a': 4,
 'abc': 5,
 'abcpolitics': 6,
 'abedin': 7,
 'about': 8,
 'aboutface': 9,
 'abuses': 10,
 'accused': 11,
 'accusing': 12,
 'act': 13,
 'ad': 14,
 'admits': 15,
 'ads': 16,
 'afraid': 17,
 'after': 18,
 'afternoon': 19,
 'against': 20,
 'age': 21,
 'agency': 22,
 'agents': 23,
 'ago': 24,
 'ahead': 25,
 'alive': 26,
 'all': 27,
 'allegations': 28,
 'allies': 29,
 'allowed': 30,
 'already': 31,
 'also': 32,
 'amendment': 33,
 'americans': 34,
 'an': 35,
 'and': 36,
 'announced': 37,
 'anthony': 38,
 'any': 39,
 'anywhere': 40,
 'apolitical': 41,
 'appearance': 42,
 'appeared': 43,
 'appearing': 44,
 'appeaser': 45,
 'approach': 46,
 'are': 47,
 'around': 48,
 'arrogant': 49,
 'article': 50,
 'as': 51,
 'asked': 52,
 'assault': 53,
 'assaulting': 54,
 'assaults': 55,
 'associates': 56,
 'assume': 57,
 'at': 58,
 'attack': 59,
 'attacked': 60,
 'attacking': 61,
 'away': 62,
 'awkward': 63,
 'awkwardly': 64,
 'back': 65,
 'backed': 66,
 'b

In [14]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i+ n_words])

In [17]:
next_words

['is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 'but',
 'it',
 's',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 'but',
 'that',
 's',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'director',
 'james',
 'comey',
 'is',
 'hillary',
 's',
 'opponent',
 'in',
 'this',
 'election',
 'the',
 'fbi',
 'is',
 'under',
 '

In [18]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [21]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1        

In [22]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [26]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X,y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.0737 - loss: 5.2280
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0914 - loss: 4.8942
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.1251 - loss: 4.5372
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1158 - loss: 4.3767
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1404 - loss: 4.1449
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1753 - loss: 3.8695
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1745 - loss: 3.8759
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2434 - loss: 3.2442
Epoch 9/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2929 - loss: 2.9825
Epoch 10/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.3663 - loss: 2.6601
Epoch 11/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4379 - loss: 2.4023
Epoch 12/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy:

In [27]:
model.save("mymodel.h5")

In [28]:
model = load_model("mymodel.h5")

In [30]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [31]:
possible = predict_next_word("He will have to look into this thing and he", 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step


In [32]:
possible

array([362, 332, 333, 417,  71], dtype=int64)

In [33]:
print([unique_tokens[idx] for idx in possible])

['made', 'knew', 'know', 'out', 'be']


In [34]:
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)                 

In [35]:
generate_text("He will have to look into this thing and he", 100, 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━

'He will have to look into this thing and he be in in smoke this than a president from from their google pinterest months until sent in just chilly afternoon a particularly afternoon trump trump trump he carville not about recover if in this want that an be from to clintons it has also to be patients and fbi often hillary s followed afraid of that a abedin and and agents t victory have afraid smell can weathered hillary their isn her credibility a whole countless their allies on the years t they americans mysterious off that her associates scandal to try a reveal on on fbi republicans was was'